In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller

# Загрузка данных (замените 'your_data.csv' на путь к вашему файлу данных)
df = pd.read_csv('your_data.csv', parse_dates=['date'], index_col='date')

# Убедитесь, что данные упорядочены по дате
df = df.sort_index()

# Выберите столбец с загруженностью станций
series = df['total_load']

# Проверка на стационарность
result = adfuller(series)
print(f'ADF Statistic: {result[0]}')
print(f'p-value: {result[1]}')


In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Обучение модели ARIMA
model_arima = ARIMA(series, order=(5, 1, 2))
model_arima_fit = model_arima.fit()

# Прогноз на два года вперед (730 дней)
forecast_steps = 730
forecast_arima = model_arima_fit.forecast(steps=forecast_steps)
forecast_index = pd.date_range(start=series.index[-1], periods=forecast_steps + 1, closed='right')
forecast_series_arima = pd.Series(forecast_arima, index=forecast_index)


In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Обучение модели SARIMA
model_sarima = SARIMAX(series, order=(5, 1, 2), seasonal_order=(1, 1, 1, 12))
model_sarima_fit = model_sarima.fit()

# Прогноз на два года вперед (730 дней)
forecast_sarima = model_sarima_fit.forecast(steps=forecast_steps)
forecast_series_sarima = pd.Series(forecast_sarima, index=forecast_index)


In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Подготовка данных для LSTM
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_series = scaler.fit_transform(series.values.reshape(-1, 1))

# Разделение на обучающую и тестовую выборки
train_size = int(len(scaled_series) * 0.8)
train, test = scaled_series[:train_size], scaled_series[train_size:]

# Создание данных для LSTM
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 10
X_train, y_train = create_dataset(train, look_back)
X_test, y_test = create_dataset(test, look_back)

# Изменение формы данных для LSTM [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Построение модели LSTM
model_lstm = Sequential()
model_lstm.add(LSTM(50, return_sequences=True, input_shape=(look_back, 1)))
model_lstm.add(LSTM(50, return_sequences=False))
model_lstm.add(Dense(1))

model_lstm.compile(optimizer='adam', loss='mean_squared_error')
model_lstm.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1)

# Прогнозирование с помощью LSTM
train_predict = model_lstm.predict(X_train)
test_predict = model_lstm.predict(X_test)

# Инвертирование предсказаний
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)

# Создание прогноза на два года вперед (730 дней)
input_sequence = scaled_series[-look_back:].reshape(1, look_back, 1)
forecast_lstm = []
for _ in range(forecast_steps):
    prediction = model_lstm.predict(input_sequence)
    forecast_lstm.append(prediction[0, 0])
    input_sequence = np.append(input_sequence[:, 1:, :], prediction.reshape(1, 1, 1), axis=1)

forecast_lstm = scaler.inverse_transform(np.array(forecast_lstm).reshape(-1, 1))
forecast_series_lstm = pd.Series(forecast_lstm.flatten(), index=forecast_index)
# Сохранение моделей и масштабировщиков
with open('models.pkl', 'wb') as model_file, open('scalers.pkl', 'wb') as scaler_file:
    pickle.dump(models, model_file)
    pickle.dump(scalers, scaler_file)

# Загрузка моделей и масштабировщиков
with open('models.pkl', 'rb') as model_file, open('scalers.pkl', 'rb') as scaler_file:
    models = pickle.load(model_file)
    scalers = pickle.load(scaler_file)